# DEXES memecoins

## Importaciones

In [1]:
from solders.keypair import Keypair
from typing import Literal, Union

from solana_manager import SolanaAccountInfo, SolanaWalletManager
from pumpfun import PumpFunTransactions, PumpFunPriceFetcher
from dexscreener import DexScreenerPriceTracker, DexScreenerPortfolioMonitor

🎯 Módulo PumpFun cargado
🚀 Funcionalidades disponibles:
   - Fetcher de precios directo de bonding curve
   - Análisis de estado de curve
   - Cálculo de progreso de bonding
   - Precios en tiempo real desde Pump.fun
   - Cliente API centralizado con WebSocket y HTTP
   - Trading con APIs Lightning y Local
   - Gestión de wallets Lightning
   - Creación de tokens
   - Soporte para API key en WebSocket (PumpSwap data)
   - Parser asíncrono de transacciones de Pump.fun
   - Análisis de actividad de traders
   - Patrón Singleton y Context Manager


## Configuraciones

In [2]:
NETWORK = "mainnet-beta"
RPC_URL = "https://api.mainnet-beta.solana.com"
WALLET_FILE = "wallets/wallet_pumpfun.json"
SLIPPAGE = 15.0
PRIORITY_FEE = 0.0001

wallet_data = None
async with SolanaWalletManager(network=NETWORK) as solana_manager:
    wallet_data = await solana_manager.load_wallet_from_file(filename=WALLET_FILE)
    wallet_data["key_pair"] = Keypair().from_base58_string(wallet_data["private_key"])

account_info = SolanaAccountInfo(network=NETWORK)
transactions = PumpFunTransactions()
pump_fetcher = PumpFunPriceFetcher()
price_tracker = DexScreenerPriceTracker()

if not wallet_data:
    raise Exception("Wallet not found")

portfolio_monitor = DexScreenerPortfolioMonitor(wallet_data.get("public_key"), price_tracker)

🌐 Conectado a Solana mainnet-beta (RPC: https://api.mainnet-beta.solana.com)
📂 Wallet cargada desde wallets/wallet_pumpfun.json
🔌 Conexión a Solana cerrada.
🌐 PumpFun API Client inicializado
   WebSocket: ❌ wss://pumpportal.fun/api/data
   HTTP: ✅ https://pumpportal.fun/api
   API Key: ❌ No proporcionada
🎯 Pump.fun Price Fetcher inicializado
🌐 Configurado para conectar a https://api.mainnet-beta.solana.com
📊 DexScreener Price Tracker inicializado (Async)
📊 DexScreener Portfolio Monitor inicializado (Async)
📍 Wallet: GMN2f6Ps...dfBtnAtz


📂 No se encontraron datos previos del portfolio


## Información

### Balance

In [3]:
async with portfolio_monitor:
    await portfolio_monitor.get_detailed_balance()


💰 BALANCE DETALLADO DE WALLET
🪙 SOL:
   Balance: 0.016694 SOL
   Precio: $164.48 USD
   Valor: $2.75 USD

🪙 TOKENS (4 encontrados):

📊 RESUMEN TOTAL:
   💰 SOL: 0.016694 SOL ($2.75 USD)
   🪙 Tokens: $0.000000 USD
   💎 TOTAL USD: $2.75
   💎 TOTAL SOL: 0.016694 SOL
🔒 DexScreener Portfolio Monitor cerrado


# Trading

In [4]:
TOKEN_MINT="6F6jjd71nbjwUZNi93wpXimbyH12CLkeMNJ6X1brpump"

### Precio de token

In [5]:
result = None
async with price_tracker:
    result = await price_tracker.get_token_price(TOKEN_MINT)

if not result:
    print("🔄 Reintentando con Pump.fun Price Fetcher")
    token_price = None
    async with pump_fetcher:
        token_price = await pump_fetcher.get_token_price(TOKEN_MINT)

    if token_price:
        print(f"\n🎉 PRECIO OBTENIDO EXITOSAMENTE!")
        print(f"💰 Precio SOL: {token_price.price_sol:.12f}")
        print(f"💵 Precio USD: ${token_price.price_usd:.12f}")
        print(f"📊 Market Cap: ${token_price.market_cap_usd:,.2f}")
        print(f"📈 Progreso Bonding: {token_price.bonding_progress:.2f}%")
        print(f"⏰ Timestamp: {token_price.timestamp}")
    else:
        print("❌ No se pudo obtener el precio")

💰 Obteniendo precio para: 6F6jjd71...
✅ Precio obtenido desde DexScreener tokens: $0.0000044260 USD
🔒 DexScreener Price Tracker cerrado


### Local trade

In [24]:
ACTION: Literal["buy", "sell"] = "sell"
AMOUNT: Union[float, str] = 0.00001
DENOMINATED_IN_SOL: bool = True
SLIPPAGE: float = 15.0
PRIORITY_FEE: float = 0.00005

async with transactions:
    signature = await transactions.create_and_send_local_trade(
        keypair=wallet_data.get("key_pair"),
        action=ACTION,
        mint=TOKEN_MINT,
        amount=AMOUNT,
        denominated_in_sol=DENOMINATED_IN_SOL,
        slippage=SLIPPAGE,
        priority_fee=PRIORITY_FEE
    )
    print(f"🔗 Transacción enviada exitosamente. Puedes verla en Solscan: https://solscan.io/tx/{signature}?cluster=mainnet-beta")

🔌 Iniciando sesión de transacciones PumpFun...
✅ Sesión HTTP inicializada
🛠️ Creando transacción local: sell 1e-05 de 6F6jjd71nbjwUZNi93wpXimbyH12CLkeMNJ6X1brpump
📄 Transacción recibida, firmando localmente...
🖋️ Transacción firmada, enviando a https://api.mainnet-beta.solana.com/...
✅ Transacción enviada. Firma: 66ioQqm668um5riAN3vdRmMHVx5znaVypCCb2iijkCtfnJKcPAfkV2xESwNnf7uhJp2PLZpTRa7admWUZfrpVHLG
🔗 Transacción enviada exitosamente. Puedes verla en Solscan: https://solscan.io/tx/66ioQqm668um5riAN3vdRmMHVx5znaVypCCb2iijkCtfnJKcPAfkV2xESwNnf7uhJp2PLZpTRa7admWUZfrpVHLG?cluster=mainnet-beta
🔌 Cerrando sesión de transacciones PumpFun...
🔌 Sesión HTTP cerrada
✅ Sesión de transacciones cerrada correctamente
